In [17]:
import torch
import torch.nn as nn
from trainer_functions.waterbirdsloader import WaterbirdsDataset
from trainer_functions.waterbirdstrainer import evaluate_waterbirds, train_waterbirds
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import wandb
wandb.init(project="Surgical_Test", name="tuneall")

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

accuracy,▇▁▁▇▂▇▇▇▇▇▇▇▇▇▁▂▇▇▅▃▇▇▇█▅▇▇▇▃▆▇█▇▇▇▇▇▇▇█
loss,█▅▆▆▄▅▅▆▅▅▅▆▅▆▅▄▃▂▂▅▂▂▂▁▂▂▂▄▂▂▁▁▂▂▂▁▁▁▁▁
accuracy,0.85232
loss,0.02572


In [18]:
wb_train = WaterbirdsDataset(csv_dir = "data/waterbird_complete95_forest2water2/metadata.csv", mode = "train", root_dir="data/waterbird_complete95_forest2water2")
wb_target = WaterbirdsDataset(csv_dir = "data/waterbird_complete95_forest2water2/metadata.csv", mode = "val", root_dir="data/waterbird_complete95_forest2water2")
wbtarget_test = WaterbirdsDataset(csv_dir = "data/waterbird_complete95_forest2water2/metadata.csv", mode = "test", root_dir="data/waterbird_complete95_forest2water2")

In [19]:
# split wbtrain dataset into train and validation
train_size = int(0.8 * len(wb_train))
val_size = len(wb_train) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(wb_train, [train_size, val_size])

# take 400 images from wb_target
target_size = 400
target_dataset, target_val = torch.utils.data.random_split(wb_target, [target_size, len(wb_target) - target_size])


In [20]:
print("Train size: ", len(train_dataset))
print("Val size: ", len(val_dataset))
print("Target size: ", len(target_dataset))
print("Target val size: ", len(target_val))
print("Target test size: ", len(wbtarget_test))

Train size:  3836
Val size:  959
Target size:  400
Target val size:  799
Target test size:  5794


In [21]:
# source loader
wb_trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
wb_valloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

# target loader
wb_targetloader = DataLoader(target_dataset, batch_size=64, shuffle=True)
wb_target_valloader = DataLoader(target_val, batch_size=64, shuffle=True)

target_testloader = DataLoader(wbtarget_test, batch_size=64, shuffle=True)

In [22]:
# get a pretrained model trained on IMAGENET
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(2048, 2) # change the last layer to 2 classes

/home/sheikh/miniconda3/envs/cnn/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/sheikh/miniconda3/envs/cnn/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [23]:
# load pretrained model_40 from model_states
df = pd.DataFrame(columns=["accuracy", "lr", "state", "n_subset"])
lrs = [0.005, 0.001, 0.0005]


In [24]:

for lr in lrs:
    wb_targetloader = DataLoader(target_dataset, batch_size=64, shuffle=True)
    wb_target_valloader = DataLoader(target_val, batch_size=64, shuffle=True)

    device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')

    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(2048, 2)

    model.load_state_dict(torch.load("model_states/model_40.pth"))

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.0001) # change the optimizer to tune different layers


    acc = train_waterbirds(model, wb_targetloader, wb_target_valloader, optimizer, scheduler=None, device=device, epochs=15)

    # adding results to dataframe 
    dict = {}
    dict['accuracy'] = acc
    dict['lr'] = lr
    dict['state'] = "all"
    dict['n_subset'] = 400
    df_temp = pd.DataFrame(dict, index=[0])
    df = pd.concat([df, df_temp]) 



/home/sheikh/miniconda3/envs/cnn/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/sheikh/miniconda3/envs/cnn/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0, loss = 0.3124
Got 625 / 799 correct (78.22)

Epoch 1, loss = 0.6658
Got 174 / 799 correct (21.78)

Epoch 2, loss = 0.6507
Got 625 / 799 correct (78.22)

Epoch 3, loss = 0.3073
Got 625 / 799 correct (78.22)

Epoch 4, loss = 0.3891
Got 619 / 799 correct (77.47)

Epoch 5, loss = 0.4582
Got 625 / 799 correct (78.22)

Epoch 6, loss = 0.6189
Got 625 / 799 correct (78.22)

Epoch 7, loss = 0.4065
Got 625 / 799 correct (78.22)

Epoch 8, loss = 1.0552
Got 625 / 799 correct (78.22)

Epoch 9, loss = 0.4649
Got 625 / 799 correct (78.22)

Epoch 10, loss = 0.8148
Got 625 / 799 correct (78.22)

Epoch 11, loss = 0.4006
Got 507 / 799 correct (63.45)

Epoch 12, loss = 0.7285
Got 618 / 799 correct (77.35)

Epoch 13, loss = 0.4320
Got 625 / 799 correct (78.22)

Epoch 14, loss = 0.3565
Got 625 / 799 correct (78.22)



/home/sheikh/miniconda3/envs/cnn/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/sheikh/miniconda3/envs/cnn/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0, loss = 0.3586
Got 563 / 799 correct (70.46)

Epoch 1, loss = 0.3543
Got 174 / 799 correct (21.78)

Epoch 2, loss = 0.5942
Got 268 / 799 correct (33.54)

Epoch 3, loss = 0.2178
Got 277 / 799 correct (34.67)

Epoch 4, loss = 0.1605
Got 624 / 799 correct (78.10)

Epoch 5, loss = 0.2697
Got 621 / 799 correct (77.72)

Epoch 6, loss = 0.0903
Got 609 / 799 correct (76.22)

Epoch 7, loss = 0.3177
Got 632 / 799 correct (79.10)

Epoch 8, loss = 0.2253
Got 631 / 799 correct (78.97)

Epoch 9, loss = 0.6731
Got 628 / 799 correct (78.60)

Epoch 10, loss = 0.1395
Got 628 / 799 correct (78.60)

Epoch 11, loss = 0.0469
Got 617 / 799 correct (77.22)

Epoch 12, loss = 0.2485
Got 613 / 799 correct (76.72)

Epoch 13, loss = 0.0617
Got 599 / 799 correct (74.97)

Epoch 14, loss = 0.0508
Got 627 / 799 correct (78.47)



/home/sheikh/miniconda3/envs/cnn/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/sheikh/miniconda3/envs/cnn/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0, loss = 0.4370
Got 670 / 799 correct (83.85)

Epoch 1, loss = 0.4608
Got 570 / 799 correct (71.34)

Epoch 2, loss = 0.2512
Got 661 / 799 correct (82.73)

Epoch 3, loss = 0.0351
Got 679 / 799 correct (84.98)

Epoch 4, loss = 0.4963
Got 683 / 799 correct (85.48)

Epoch 5, loss = 0.0434
Got 668 / 799 correct (83.60)

Epoch 6, loss = 0.0840
Got 689 / 799 correct (86.23)

Epoch 7, loss = 0.0281
Got 652 / 799 correct (81.60)

Epoch 8, loss = 0.4798
Got 628 / 799 correct (78.60)

Epoch 9, loss = 0.0341
Got 608 / 799 correct (76.10)

Epoch 10, loss = 0.0732
Got 644 / 799 correct (80.60)

Epoch 11, loss = 0.0202
Got 662 / 799 correct (82.85)

Epoch 12, loss = 0.0046
Got 680 / 799 correct (85.11)

Epoch 13, loss = 0.0048
Got 699 / 799 correct (87.48)

Epoch 14, loss = 0.1583
Got 702 / 799 correct (87.86)



In [ ]:
df.to_csv("waterbirdtuneall.csv")